In [1]:
targetColName="like"
targetIndex=3
lr=[0.05,0.03,0.07,0.01]
origin="drive/MyDrive/twitter/onechunk/"

In [2]:
!pip uninstall scikit-learn

Uninstalling scikit-learn-0.22.2.post1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/scikit_learn-0.22.2.post1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-0.22.2.post1


In [3]:
!pip install 'scikit-learn>=0.24'

     |████████████████████████████████| 22.3MB 1.7MB/s 


In [4]:
import sklearn
sklearn.__version__

'0.24.2'

#Prepare for training

In [5]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold

In [6]:
from sklearn.metrics import mean_squared_error

import gc
gc.collect()

50

In [7]:

import os   
os.system(f"cp {origin}*.csv ../content/")

0

In [8]:
import pandas as pd

In [9]:
train=pd.read_csv("trainFinal.csv", index_col=0)
test=pd.read_csv("testFinal.csv", index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [10]:
test.columns==train.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [11]:
columns=["retweet","retweetCom","reply","like"]

trainTarget= train[targetColName]
testTarget= test[targetColName]

for col in columns:
    train.drop(col,axis=1,inplace=True)
    test.drop(col,axis=1,inplace=True)
    

##Get data in correct format

In [12]:
trainXGB=train.values
trainTargetXGB=trainTarget.values
del train
del trainTarget

In [13]:
testXGB=test.values
testTargetXGB=testTarget.values
del test
del testTarget

#Model Building

##HistGradientBoostingClassifier computation

In [14]:
weigths=trainTargetXGB.copy()

In [15]:
for i in range(len(weigths)):
    if weigths[i]==1:
        weigths[i]=2.5
    else:
        weigths[i]=1

In [16]:
from sklearn.experimental import enable_hist_gradient_boosting  

from sklearn.ensemble import HistGradientBoostingClassifier

In [17]:
HistGradientBoostingClassifier()

HistGradientBoostingClassifier()

In [18]:
model = HistGradientBoostingClassifier(loss="binary_crossentropy",learning_rate=0.1,max_depth=16,l2_regularization=0.001,max_leaf_nodes=511,early_stopping=True,
                                       validation_fraction=0.1,verbose=1,random_state=1234,max_bins=255,max_iter=100)

In [19]:
trained=model.fit(trainXGB, trainTargetXGB,weigths)

1 tree, 511 leaves, max depth = 16, train loss: 0.22311, val loss: 0.23316, in 0.607s
[98/100] 1 tree, 218 leaves, max depth = 16, train loss: 0.22307, val loss: 0.23316, in 0.673s
[99/100] 1 tree, 510 leaves, max depth = 16, train loss: 0.22295, val loss: 0.23311, in 0.756s
[100/100] 1 tree, 511 leaves, max depth = 16, train loss: 0.22274, val loss: 0.23297, in 0.779s
Fit 100 trees in 108.961 s, (50530 total leaves)
Time spent computing histograms: 49.573s
Time spent finding best splits:  2.262s
Time spent applying splits:      9.703s
Time spent predicting:           1.207s


In [20]:
trainPredProba = trained.decision_function(trainXGB)
fpr_gb, tpr_gb, _ = roc_curve(trainTargetXGB, trainPredProba)
roc_auc_gb = auc(fpr_gb, tpr_gb)

In [21]:
print("Area under ROC curve = {:0.2f}".format(roc_auc_gb))

Area under ROC curve = 0.95


In [22]:
from sklearn.metrics import roc_auc_score
y = trainTargetXGB
auc_cr = roc_auc_score(y,trainPredProba) 
print("cortado auc: {auc_cr}".format(auc_cr=auc_cr))

cortado auc: 0.945591888960825


In [23]:
testPred=trained.predict(testXGB)

In [24]:
cm = confusion_matrix(testTargetXGB,testPred)
cm

array([[352868,  13168],
       [219155,  22044]])

In [25]:
print(classification_report(testTargetXGB,testPred))

              precision    recall  f1-score   support

           0       0.62      0.96      0.75    366036
           1       0.63      0.09      0.16    241199

    accuracy                           0.62    607235
   macro avg       0.62      0.53      0.46    607235
weighted avg       0.62      0.62      0.52    607235



In [26]:
testPredProba = trained.decision_function(testXGB)
fpr_gb, tpr_gb, _ = roc_curve(testTargetXGB, testPredProba)
roc_auc_gb = auc(fpr_gb, tpr_gb)

In [27]:
print("Area under ROC curve = {:0.2f}".format(roc_auc_gb))

Area under ROC curve = 0.54


In [28]:
from sklearn.metrics import roc_auc_score
y = testTargetXGB
auc_cr = roc_auc_score(y,testPredProba) 
print("cortado auc: {auc_cr}".format(auc_cr=auc_cr))

cortado auc: 0.5369805297439184


In [29]:
from sklearn.metrics import average_precision_score, log_loss

def calculate_ctr(gt):
  positive = len([x for x in gt if x == 1])
  ctr = positive/float(len(gt))
  return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

rce = compute_rce(testPredProba, y)
average_precision = average_precision_score(y,testPredProba)

In [30]:
rce

-1832.7733652331372

In [31]:
average_precision

0.4537993331872621